Generate the baseline scores for drug response prediction.<br>
The baseline is when we use cell line features and drug labels.
At this point I tried to use both labels of cells and drugs. This didn't work at all.
TODO: Update this code to use cell features and drug labels.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
from pathlib import Path
import sys
from time import time
import numpy as np
import pandas as pd

import sklearn
from collections import OrderedDict
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

SEED = None

## Choose SOURCE

In [2]:
# SOURCE = 'ccle'
# SOURCE = 'gdsc'
SOURCE = 'ctrp'
dirpath = Path(f'../../data/processed/data_splits/{SOURCE}_cv_simple')

In [3]:
# xdata = pd.read_parquet( dirpath/'xdata.parquet', engine='auto', columns=None )
# ydata = pd.read_parquet( dirpath/'ydata.parquet', engine='auto', columns=None )
meta = pd.read_parquet( dirpath/'meta.parquet', engine='auto', columns=None )

In [4]:
Y = meta[['AUC']].copy()
X = meta[['CELL', 'DRUG']].copy()

print(X.shape)
print(Y.shape)
X[:3]

(324952, 2)
(324952, 1)


,CELL,DRUG
0,CTRP.22RV1,CTRP.1
1,CTRP.23132-87,CTRP.1
2,CTRP.253J,CTRP.1


### Encode labels

In [5]:
# encoding_method = 'label'
encoding_method = 'onehot'

In [6]:
if encoding_method == 'label':
    from sklearn.preprocessing import LabelEncoder
    X_enc = X.copy()

    # Label encoder
    cell_enc = LabelEncoder()
    X_enc['CELL'] = cell_enc.fit_transform(X_enc['CELL'].values)

    drug_enc = LabelEncoder()
    X_enc['DRUG'] = drug_enc.fit_transform(X_enc['DRUG'].values)

    X = X_enc
    print(X.shape)
    display(X[:3])

    
elif encoding_method == 'onehot':
    from sklearn.preprocessing import OneHotEncoder
    X_onehot = X.copy()

    # Onehot encoder
    cell_enc = OneHotEncoder(sparse=False)
    cell = cell_enc.fit_transform(X_onehot['CELL'].values.reshape(-1, 1))
    print(cell.shape)

    drug_enc = OneHotEncoder(sparse=False)
    drug = drug_enc.fit_transform(X_onehot['DRUG'].values.reshape(-1, 1))
    print(drug.shape)

    # from scipy.sparse import hstack
    # X_onehot = hstack((cell, drug))
    # print(X_onehot.shape)

    X_onehot = pd.DataFrame( np.concatenate((cell, drug), axis=1) )
    
    X = X_onehot
    print(X.shape)
    display(X[:3])

(324952, 810)
(324952, 495)
(324952, 1305)


,0,1,2,3,4,5,6,7,8,9,...,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Load CV partitions

In [7]:
folds = 5
tr_id = pd.read_csv(dirpath/f'{folds}fold_tr_id.csv')
vl_id = pd.read_csv(dirpath/f'{folds}fold_vl_id.csv')

cv_splits = (tr_id, vl_id)
del tr_id, vl_id

In [8]:
Y = Y.values
X = X.values

tr_dct = {}
vl_dct = {}

tr_id = cv_splits[0]
vl_id = cv_splits[1]

for i in range(tr_id.shape[1]):
    tr_dct[i] = tr_id.iloc[:, i].dropna().values.astype(int).tolist()
    vl_dct[i] = vl_id.iloc[:, i].dropna().values.astype(int).tolist()

### ML models

In [9]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
# https://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html

from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

model_name = 'elastic_net'
# model_name = 'svr'
# model_name = 'krnl_ridge'

def get_model(model_name):
    if model_name == 'elastic_net':
        model = ElasticNet(
            alpha=1.0, l1_ratio=0.5, fit_intercept=True, normalize=False, precompute=False, max_iter=1000, copy_X=True,
            tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')

    elif model_name == 'svr':
        model = SVR(
            kernel='rbf', degree=3, gamma='auto_deprecated', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True,
            cache_size=200, verbose=False, max_iter=-1)

    elif model_name == 'krnl_ridge':
        model = KernelRidge(
            alpha=1, kernel='linear', gamma=None, degree=3, coef0=1, kernel_params=None)
        
    return model

## sklearn CV

In [13]:
# idx_new = np.random.permutation(data.shape[0])
# data_shf = data[idx_new, :]

data = pd.DataFrame( np.concatenate((Y, X), axis=1) )

data_shf = data.sample(frac=1.0).reset_index(drop=True)
ydata = data_shf.iloc[:,0].values
xdata = data_shf.iloc[:,1:].values

In [14]:
# display(sorted(sklearn.metrics.SCORERS.keys()))

In [15]:
from sklearn.model_selection import cross_validate, cross_val_score

model = get_model(model_name)
cross_validate(estimator=model, X=xdata, y=ydata, scoring='r2', cv=folds, n_jobs=8, verbose=True)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   25.1s remaining:   37.7s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   25.6s finished


{'fit_time': array([19.21634889, 19.29753494, 19.32204723, 19.30689478, 19.36246896]),
 'score_time': array([0.1252172 , 0.12403798, 0.12683535, 0.12477875, 0.12757039]),
 'test_score': array([-6.28185303e-06, -3.54119868e-05, -1.68242857e-04, -3.62868402e-06,
        -1.25426981e-04]),
 'train_score': array([0., 0., 0., 0., 0.])}

### CV train

In [10]:
file_path = Path(os.getcwd())
print(file_path)
utils_path = file_path / '../../utils'
sys.path.append(str(utils_path))
import utils

/vol/ml/apartin/projects/cell-line-drug-sensitivity/apps/lrn_crv


In [ ]:
# Now start nested loop of train size and cv folds
tr_scores_all = [] # list of dicts
vl_scores_all = [] # list of dicts

# CV loop
for fold, (tr_k, vl_k) in enumerate(zip( tr_dct.keys(), vl_dct.keys() )):
    print(f'\nFold {fold}')
    tr_id = tr_dct[tr_k]
    vl_id = vl_dct[vl_k]
    
    # Samples from this dataset are sampled for training
    xtr = X[tr_id, :]
    ytr = Y[tr_id, :]
    print(   'xtr.shape', xtr.shape)

    # A fixed set of validation samples for the current CV split
    xvl = X[vl_id, :]
    yvl = np.squeeze(Y[vl_id, :])
    print(   'xvl.shape', xvl.shape)

    # Define and train ML model
    model = get_model(model_name)
    model.fit(xtr, ytr)
    ytr_pred = model.predict(xtr)
    yvl_pred = model.predict(xvl)
    
    tr_scores = utils.calc_scores(y_true=ytr, y_preds=ytr_pred, mltype='reg', metrics=None)
    vl_scores = utils.calc_scores(y_true=yvl, y_preds=yvl_pred, mltype='reg', metrics=None)

    # Add info
    tr_scores['tr_set'] = True
    vl_scores['tr_set'] = False
    tr_scores['fold'] = 'f'+str(fold)
    vl_scores['fold'] = 'f'+str(fold)

    # Aggregate scores
    tr_scores_all.append(tr_scores)
    vl_scores_all.append(vl_scores)
    


Fold 0
xtr.shape (292456, 2)
xvl.shape (32496, 2)


/vol/ml/apartin/anaconda3/envs/p1/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
model.coef_.shape
# model.dual_coef_

(1305,)

In [14]:
def scores_to_df(scores_all):
    df = pd.DataFrame(scores_all)
    df = df.melt(id_vars=['fold', 'tr_set'])
    df = df.rename(columns={'variable': 'metric'})
    df = df.pivot_table(index=['metric', 'tr_set'], columns=['fold'], values='value')
    df = df.reset_index(drop=False)
    df.columns.name = None
    return df

In [15]:
tr_df = scores_to_df(tr_scores_all)
vl_df = scores_to_df(vl_scores_all)

In [16]:
tr_df

,metric,tr_set,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9
0,auroc_reg,True,5.000000e-01,5.000000e-01,5.000000e-01,5.000000e-01,5.000000e-01,5.000000e-01,5.000000e-01,5.000000e-01,5.000000e-01,5.000000e-01
1,mean_absolute_error,True,1.081672e-01,1.083913e-01,1.083266e-01,1.081980e-01,1.081416e-01,1.081291e-01,1.082123e-01,1.082364e-01,1.081450e-01,1.081892e-01
2,mean_squared_error,True,2.082718e-02,2.091587e-02,2.087860e-02,2.084982e-02,2.081677e-02,2.081818e-02,2.084995e-02,2.087111e-02,2.083595e-02,2.084070e-02
3,median_absolute_error,True,9.416085e-02,9.434468e-02,9.429869e-02,9.426283e-02,9.417225e-02,9.412562e-02,9.422815e-02,9.418374e-02,9.416037e-02,9.417473e-02
4,r2,True,-8.406877e-10,-1.778336e-09,-1.220031e-09,1.096974e-09,-3.971987e-10,1.288251e-09,-3.055598e-10,-1.028357e-10,-9.360668e-10,7.925688e-10
